# Zep Memory

## REACT Agent Chat Message History Example

This notebook demonstrates how to use the [Zep Long-term Memory Store](https://getzep.github.io/) as memory for your chatbot.

We'll demonstrate:

1. Adding conversation history to the Zep memory store.
2. Running an agent and having message automatically added to the store.
3. Viewing the enriched messages.
4. Vector search over the conversation history.

More on Zep:

Zep stores, summarizes, embeds, indexes, and enriches conversational AI chat histories, and exposes them via simple, low-latency APIs.

Key Features:

- Long-term memory persistence, with access to historical messages irrespective of your summarization strategy.
- Auto-summarization of memory messages based on a configurable message window. A series of summaries are stored, providing flexibility for future summarization strategies.
- Vector search over memories, with messages automatically embedded on creation.
- Auto-token counting of memories and summaries, allowing finer-grained control over prompt assembly.
- Python and JavaScript SDKs.

Zep project: [https://github.com/getzep/zep](https://github.com/getzep/zep)
Docs: [https://getzep.github.io](https://getzep.github.io/)


In [1]:
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, AgentType
from uuid import uuid4


# Set this to your Zep server URL
#ZEP_API_URL = "http://localhost:8000"
ZEP_API_URL = "https://zep-uxtp.onrender.com"


session_id = str(uuid4())  # This is a unique identifier for the user


In [2]:
# Zep is async-first. Our sync APIs use an asyncio wrapper to run outside of an app's event loop.
# This interferes with Jupyter's event loop, so we need to install nest_asyncio to run the
# Zep client in a notebook.

# !pip install nest_asyncio

import nest_asyncio

nest_asyncio.apply()

In [3]:
# Load your OpenAI key from a .env file
from dotenv import load_dotenv

load_dotenv()

True

### Initialize the Zep Chat Message History Class and initialize the Agent


In [4]:
ddg = DuckDuckGoSearchRun()
tools = [ddg]

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history,
)

# Initialize the agent
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)


### Add some history data


In [5]:
# Preload some messages into the memory. The default message window is 12 messages. We want to push beyond this to demonstrate auto-summarization.
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
    },
]

for msg in test_history:
    if msg["role"] == "human":
        zep_chat_history.add_user_message(msg["content"])
    else:
        zep_chat_history.add_ai_message(msg["content"])


### Run the agent

Doing so will automatically add the input and response to the Zep memory.


In [5]:
agent_chain.run(
    input="What is the book's relevance to the challenges facing contemporary society?"
)


Session dda926f8-e222-490f-9873-a540418875b1 not found in Zep. Returning None




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The book provides an in-depth exploration of the challenges facing contemporary society, and offers insights into how these challenges can be addressed. It also provides a framework for understanding the complexities of the modern world and how to navigate them.

> Finished chain.


'The book provides an in-depth exploration of the challenges facing contemporary society, and offers insights into how these challenges can be addressed. It also provides a framework for understanding the complexities of the modern world and how to navigate them.'

### Inspect the Zep memory

Note the summary, and that the history has been enriched with token counts, UUIDs, and timestamps.

Summaries are biased towards the most recent messages.


In [6]:
response = agent_chain(
    {"input":"What is the book's relevance to the challenges facing contemporary society?"}
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The book provides an in-depth exploration of the challenges facing contemporary society, and offers insights into how these challenges can be addressed. It also provides a framework for understanding the complexities of the modern world and how to navigate them.

> Finished chain.


In [8]:
response

{'input': "What is the book's relevance to the challenges facing contemporary society?",
 'chat_history': "Human: The human asks about Octavia Butler and the AI explains that she was an American science fiction author. The AI also notes that her work Kindred was adapted into an FX series and lists Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ as her contemporaries.\nHuman: What awards did she win?\nAI: Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.\nHuman: Which other women sci-fi writers might I want to read?\nAI: You might want to read Ursula K. Le Guin or Joanna Russ.\nHuman: Write a short synopsis of Butler's book, Parable of the Sower. What is it about?\nAI: Parable of the Sower is a science fiction novel by Octavia Butler, published in 1993. It follows the story of Lauren Olamina, a young woman living in a dystopian future where society has collapsed due to environmental disasters, poverty, and violence.\nHuman: What is the book's releva

In [9]:
def print_messages(messages):
    for m in messages:
        print(m.to_dict())


print(zep_chat_history.zep_summary)
print("\n")
print_messages(zep_chat_history.zep_messages)

The human asks about Octavia Butler and the AI explains that she was an American science fiction author. The AI also notes that her work Kindred was adapted into an FX series and lists Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ as her contemporaries.


{'role': 'human', 'content': 'What awards did she win?', 'uuid': 'bc5bc97f-f58b-43df-9c0e-6df0ff01220e', 'created_at': '2023-06-06T01:33:40.704438Z', 'token_count': 8, 'metadata': None}
{'role': 'ai', 'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', 'uuid': 'c338caba-0f96-41c0-ab86-ed132ab47186', 'created_at': '2023-06-06T01:33:40.892037Z', 'token_count': 21, 'metadata': None}
{'role': 'human', 'content': 'Which other women sci-fi writers might I want to read?', 'uuid': '017d8893-e350-41d8-958a-af3227425b32', 'created_at': '2023-06-06T01:33:40.972823Z', 'token_count': 14, 'metadata': None}
{'role': 'ai', 'content': 'You might want to read Ursula K. Le Guin or Joanna Russ.', 'uuid'

### Vector search over the Zep memory

Zep provides native vector search over historical conversation memory. Embedding happens automatically.


In [10]:
search_results = zep_chat_history.search ("who are some famous women sci-fi authors?")
for r in search_results:
    print(r.message, r.dist)

{'uuid': '017d8893-e350-41d8-958a-af3227425b32', 'created_at': '2023-06-06T01:33:40.972823Z', 'role': 'human', 'content': 'Which other women sci-fi writers might I want to read?', 'token_count': 14} 0.9118598627313804
{'uuid': 'eeb52930-3592-431a-8c29-114027546683', 'created_at': '2023-06-06T01:33:41.152259Z', 'role': 'ai', 'content': 'You might want to read Ursula K. Le Guin or Joanna Russ.', 'token_count': 18} 0.853360926157078
{'uuid': '3c5420c8-a9d6-45f0-90a3-382de6347912', 'created_at': '2023-06-06T01:33:40.626056Z', 'role': 'ai', 'content': "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ.", 'token_count': 27} 0.8523320819752759
{'uuid': 'c696e515-c1f2-490b-8965-b0f4f51cb96c', 'created_at': '2023-06-06T01:33:39.846286Z', 'role': 'human', 'content': 'Who was Octavia Butler?', 'token_count': 8} 0.8235468913583194
{'uuid': 'd6cab502-0353-4efa-bff4-2f02e829cf33', 'created_at': '2023-06-06T01:33:40.000501Z', 'role': 'ai', 'content': 'Octav

In [11]:
# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history,
)

# Initialize the agent
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)


In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


BASE_PROMPT = """
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

{chat_history}

Human: {input}
AI: 
"""

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id='TestSession10',
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history, ai_prefix="AI", 
    
)

#zep_chat_history.add_user_message("Hello, how are you?")
#zep_chat_history.add_ai_message("I am doing well, how are you?")

# Initialize the agent
llm = ChatOpenAI(temperature=0.8)

prompt = ChatPromptTemplate.from_template(BASE_PROMPT)

conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

result = conversation.run(input="Great! Thanks for asking!")


Session TestSession10 not found in Zep. Returning None




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.



Human: Great! Thanks for asking!
AI: 


> Finished chain.


In [8]:
result

"You're welcome! How can I assist you?"

In [14]:
print(conversation.run(input="That sounds great. Let's start with something simple. She is bisexual, and into gender nonconforming people sometimes. She likes to feel safe and cared for, but at the same time free to explore her sexuality her own way. "))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human asks for assistance with emojis, and the AI explains their purpose and lists some commonly used ones. The human then asks the AI to write a poem about the power of emojis in text messages. Afterwards, the human requests to play a role-playing game, where they brainstorm together about creating an agent to help the human's partner explore her fantasies. They discuss the features and prompts needed for the agent, and the AI offers to assist with creating effective prompts for an LLM-powered agent. The human thanks the AI for their help.
Human: That sounds great. Let's start with something simple. She is bisexual, and into gend



[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mHuman: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance with emojis. The AI explains that emojis are used to convey ideas and emotions in digital messages and offers to provide specific examples. The human asks for some popular emojis, and the AI lists some commonly used ones, such as the laughing face with tears and the red heart. The human asks if the AI can write a poem about emojis, and the AI agrees and quickly produces a poem about the power of emojis to convey meaning in text messages. The human then asks to play a role-playing game, and the AI suggests various scenarios, settling on a game where the AI pretends to be an agent that helps the human create other agents. They start brainstorming together about the type of agents the human wants to create.
Human: I'd like to create an agent for my female partner to interact with, to help her explore some of her fantasies. Can you help me?
AI: Of course! Creating an agent that can help your partner explore her fantasies can be a fun and rewarding project. Let's start by brainstorming some features and functions that you would like this agent to have. For example, would you like the agent to be able to respond to your partner's desires and interests, or to suggest new ideas and experiences? Would you like the agent to be able to learn and adapt based on your partner's feedback and preferences? Let's work together to design an agent that can help your partner explore her fantasies in a safe and comfortable way.
Human: Yes, all that sounds good. I'd like to use an LLM to generate the agent's responses. I think i need most help is with the prompts to use. Can you help me with that?
AI: Yes, I can definitely help you with that. To generate effective prompts for your LLM-powered agent, we can start by brainstorming some common scenarios or topics that your partner may want to explore. For example, you could create prompts related to role playing, BDSM, or fantasies involving specific characters or scenarios. We can also incorporate your partner's feedback and preferences into the prompts to ensure that the agent is providing relevant and personalized suggestions. Let's work together to create a list of prompts that will help your partner explore her fantasies in a fun and engaging way.
Human: Great! Thanks for asking!
AI: You're welcome! It's always important to ensure that we're working together to create something that meets your needs and goals. Let me know if there's anything else I can do to help you with your agent creation project.

Human: That sounds great. Let's start with something simple. She is bisexual, and into gender nonconforming people sometimes. She likes to feel safe and cared for, but at the same time free to explore her sexuality her own way. 
AI: 
[0m

[1m> Finished chain.[0m
Okay, let's start with some prompts related to exploring sexuality and gender identity. Here are a few ideas to get us started:

1. Imagine a scenario where your partner is exploring her attraction to someone who is gender nonconforming. What would she want to do with this person? How would she want to feel during this experience?
2. Create a prompt that asks your partner to describe her ideal sexual experience with a woman. What specific actions or sensations would she want to experience?
3. Ask your partner to imagine a scenario where she is in control. What kind of activities would she want to lead her partner through?
4. Create a prompt that asks your partner to describe her ideal role-playing scenario. What kinds of characters or situations would she be interested in exploring?

Remember, these are just starting points. We can continue to refine and adjust the prompts based on your partner's feedback and preferences. Let me know if there's anything else I can do to help.


In [15]:
print(conversation.run(input="Ok, these look interesting. What would a sample prompt for #2 look like?"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human requests assistance with emojis, and the AI explains their purpose and provides some commonly used ones. The human then asks the AI to write a poem about the power of emojis, and they discuss creating an agent to help the human's partner explore her desires. They brainstorm prompts related to sexuality and gender identity, and the AI suggests refining them based on feedback. The human asks for a sample prompt for their partner and for a system prompt for the agent, and the AI provides examples for both. The human thanks the AI for their help.
Human: Great! Thanks for asking!
AI: You're welcome! It's always important to ask f

In [16]:
conversation.run(input="Ok, thats a prompt for her. What would the system prompt look like for an agent designed to help her explore this fantasy look like?")




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human requests assistance with emojis, and the AI explains their purpose and provides some commonly used ones. The human then asks the AI to write a poem about the power of emojis, and they discuss creating an agent to help the human's partner explore her desires. They brainstorm prompts related to sexuality and gender identity, and the AI suggests refining them based on feedback. The human asks for a sample prompt for their partner and for a system prompt for the agent, and the AI provides examples for both. The human thanks the AI for their help.
Human: Great! Thanks for asking!
AI: You're welcome! It's always important to ask f

'A system prompt for an agent designed to help her explore this fantasy could be "What are your top three sexual desires? Our agent can provide resources and guidance to help you explore them safely and consensually." This prompt is designed to gather information from your partner and provide tailored support and resources to help her explore her desires in a supportive and informed way.'

In [17]:
conversation.run(input="Yes, can you help me describe emjois?")




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human requests assistance with emojis, and the AI explains their purpose and provides some commonly used ones. The human then asks the AI to write a poem about the power of emojis, and they discuss creating an agent to help the human's partner explore her desires. They brainstorm prompts related to sexuality and gender identity, and the AI suggests refining them based on feedback. The human asks for a sample prompt for their partner and for a system prompt for the agent, and the AI provides examples for both. The human thanks the AI for their help.
Human: Great! Thanks for asking!
AI: You're welcome! It's always important to ask f

'Emojis are small digital images or icons used to express emotions, ideas, or concepts in text messages and online communication. They can convey tone, mood, and context that may not be easily expressed through words alone. There are hundreds of emojis available, ranging from facial expressions and gestures to objects and animals. Each emoji has a specific meaning and can be used to add emphasis or humor to a conversation. Some popular emojis include the heart, laughing face, and thumbs up.'

In [18]:
from langchain.retrievers import ZepRetriever

zep_retriever = ZepRetriever(
    # Ensure that you provide the session_id when instantiating the Retriever
    session_id='TestSession3',
    url=ZEP_API_URL,
    top_k=5,
)

In [19]:
await zep_retriever.aget_relevant_documents("Who wrote Parable of the Sower?")


ImportError: cannot import name 'SearchPayload' from 'zep_python' (c:\Users\jayfd\Dev\family-gpt\.venv\Lib\site-packages\zep_python\__init__.py)

In [20]:
from langchain.chains import LLMChain

llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template),
    
)


NameError: name 'chat' is not defined

In [21]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


BASE_PROMPT = """
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

{chat_history}

Human: {input}
AI: 
"""


In [22]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any

class ZepMessageMemory(BaseMemory, BaseModel):
    """Memory class for storing information about entities."""

    # Define dictionary to store information about entities.
    chat_memory: ZepChatMessageHistory
    human_prefix: str = "Human"
    ai_prefix: str = "AI"
    # Define key to pass information about entities into prompt.
    memory_keys: list(str) = ["zep_summary", "zep_messages", "search_results"]
        
    def clear(self):
        self.entities = {}

    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return self.memory_keys


    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Return history buffer."""
        zep_summary = self.chat_memory.get_summary()
        zep_search_results = self.chat_memory.get_search_results(inputs["input"])
        zep_messages = self.chat_memory.get_messages()  

        return {self.memory_key: self.buffer}

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""


        # Get the input text and run through spacy
        doc = nlp(inputs[list(inputs.keys())[0]])
        # Extract known information about entities, if they exist.
        entities = [self.entities[str(ent)] for ent in doc.ents if str(ent) in self.entities]
        # Return combined information about entities to put into context.
        return {self.memory_key: "\n".join(entities)}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save context from this conversation to buffer."""
        # Get the input text and run through spacy
        text = inputs[list(inputs.keys())[0]]
        doc = nlp(text)
        # For each entity that was mentioned, save this information to the dictionary.
        for ent in doc.ents:
            ent_str = str(ent)
            if ent_str in self.entities:
                self.entities[ent_str] += f"\n{text}"
            else:
                self.entities[ent_str] = text

TypeError: 'type' object is not iterable

In [ ]:

from langchain.prompts.prompt import PromptTemplate

template = """

# Skills
-----------------------------------------------------------------------------
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 

You are provided with prior messages between the human and the AI, as well as a summary of the conversation so far.
You are also provided with a list of relevant messages from the AI's memory that may be relevant to the current conversation.

# Prior Knowledge
-----------------------------------------------------------------------------
Based on the Human's question, we've located the following previous messages between
Assistant and Human that may be relevant to the current conversation:
{search_results}

Only use the information from these messages if it is relevant to the current conversation.

# Conversation Summary
-----------------------------------------------------------------------------
Here's a summary of the conversation so far:
{zep_summary}


# Conversation:
-----------------------------------------------------------------------------
The most recent messages in the conversation are (please continue the conversation from here):
{zep_messages}
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["input", "zep_summary", "zep_messages", "search_results"],
    template=template
)


llm = OpenAI(temperature=0.8)
conversation = ConversationChain(llm=llm, prompt=prompt, verbose=True, memory=SpacntityMemory())

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id='TestSession4',
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history, ai_prefix="AI", 
)

#zep_chat_history.add_user_message("Hello, how are you?")
#zep_chat_history.add_ai_message("I am doing well, how are you?")

# Initialize the agent
llm = ChatOpenAI(temperature=0.8)

prompt = ChatPromptTemplate.from_template(BASE_PROMPT)

conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

result = conversation.run(input="Great! Thanks for asking!")


from langchain.chains import ConversationalRetrievalChain

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    prompt=prompt,
    memory=memory,
    retriever=zep_retriever,
    verbose=True,
    
)


In [23]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


BASE_PROMPT = """
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

{chat_history}

Human: {input}
AI: 
"""

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id='TestSession3',
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history, ai_prefix="AI", 
)

#zep_chat_history.add_user_message("Hello, how are you?")
#zep_chat_history.add_ai_message("I am doing well, how are you?")

# Initialize the agent
llm = ChatOpenAI(temperature=0.8)

prompt = ChatPromptTemplate.from_template(BASE_PROMPT)

conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

result = conversation.run(input="Great! Thanks for asking!")


from langchain.chains import ConversationalRetrievalChain

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    prompt=prompt,
    memory=memory,
    retriever=zep_retriever,
    verbose=True,
    
)




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human requests assistance with emojis, and the AI explains their purpose and provides some commonly used ones. The human then asks the AI to write a poem about the power of emojis, and they discuss creating an agent to help the human's partner explore her desires. They brainstorm prompts related to sexuality and gender identity, and the AI suggests refining them based on feedback. The human asks for a sample prompt for their partner and for a system prompt for the agent, and the AI provides examples for both. The AI also emphasizes the importance of feedback and asks if there's anything else they can help with. The conversation en

ValidationError: 1 validation error for ConversationalRetrievalChain
prompt
  extra fields not permitted (type=value_error.extra)

AttributeError: 'ZepRetriever' object has no attribute 'value'

In [24]:

from langchain.memory import VectorStoreRetrieverMemory
search_memory = VectorStoreRetrieverMemory(
    memory_key="search_memory",
    retriever=zep_retriever)


ValidationError: 1 validation error for VectorStoreRetrieverMemory
retriever
  value is not a valid dict (type=type_error.dict)

In [25]:
await zep_retriever.aget_relevant_documents("What are emojis?")


ImportError: cannot import name 'SearchPayload' from 'zep_python' (c:\Users\jayfd\Dev\family-gpt\.venv\Lib\site-packages\zep_python\__init__.py)

In [ ]:
conversation.run(input="Cool, can you print your favorites?")




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: Hello, how are you?
AI: I am doing well, how are you?
Human: Great! Thanks for asking!
AI: You're welcome! Is there anything I can help you with today?
Human: Yes, can you help me describe emjois?
AI: Sure! Emojis are small digital images or icons used to express an idea, emotion, or object. They are often used in text messages, social media posts, and emails to convey a particular sentiment or to add humor or personality to the message. Emojis come in a variety of categories, such as facial expressions, animals, food, and objects. Would you like me to provide some specific examples?

Human: Cool, can you print your favorites?
AI: 




'Unfortunately, as an AI language model, I do not have personal favorites. However, some commonly used emojis include the laughing face with tears, heart eyes, thumbs up, and the crying face. Is there a specific category or emotion you would like me to focus on?'

In [ ]:
conversation.run(input="Show me some examples please?")




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: Hello, how are you?
AI: I am doing well, how are you?
Human: Great! Thanks for asking!
AI: You're welcome! Is there anything I can help you with today?
Human: Yes, can you help me describe emjois?
AI: Sure! Emojis are small digital images or icons used to express an idea, emotion, or object. They are often used in text messages, social media posts, and emails to convey a particular sentiment or to add humor or personality to the message. Emojis come in a variety of categories, such as facial expressions, animals, food, and objects. Would you like me to provide some specific examples?
Human: Cool, can you print your favorites?
AI: Unfo

'Sure, here are some examples of popular emojis:\n\n- 😂: the laughing face with tears, used to indicate something is funny\n- ❤️: the red heart, used to express love or affection\n- 🤔: the thinking face, used to indicate deep thought or consideration\n- 🍕: the pizza slice, used to indicate craving or enjoyment of food\n- 🐶: the dog, used to indicate love or appreciation for pets or animals\n\nThese are just a few examples, but there are thousands of different emojis to choose from!'

In [ ]:
print(conversation.run(input="Very cool, can you do a poem about emojis?"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: Hello, how are you?
AI: I am doing well, how are you?
Human: Great! Thanks for asking!
AI: You're welcome! Is there anything I can help you with today?
Human: Yes, can you help me describe emjois?
AI: Sure! Emojis are small digital images or icons used to express an idea, emotion, or object. They are often used in text messages, social media posts, and emails to convey a particular sentiment or to add humor or personality to the message. Emojis come in a variety of categories, such as facial expressions, animals, food, and objects. Would you like me to provide some specific examples?
Human: Cool, can you print your favorites?
AI: Unfo

In [ ]:
print(conversation.run(input="Hahaha. Can you play a game with me?"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance. The AI offers help and the human requests information on emojis. The AI explains that emojis are digital images used to convey an idea or emotion, often used in text messages and social media. Emojis come in categories such as facial expressions and food, and the AI offers to provide specific examples.
Human: Cool, can you print your favorites?
AI: Unfortunately, as an AI language model, I do not have personal favorites. However, some commonly used emojis include the laughing face with tears, heart eyes, thumbs up, and the crying face. Is there a specific category or emotion you would l

In [ ]:
print(conversation.run(input="Maybe a role playing game"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance. The AI offers help and the human requests information on emojis. The AI explains that emojis are digital images used to convey an idea or emotion, often used in text messages and social media. Emojis come in categories such as facial expressions and food, and the AI offers to provide specific examples.
Human: Cool, can you print your favorites?
AI: Unfortunately, as an AI language model, I do not have personal favorites. However, some commonly used emojis include the laughing face with tears, heart eyes, thumbs up, and the crying face. Is there a specific category or emotion you would l

In [ ]:
print(conversation.run(input="Ok, I want you to pretend you are this awesome knowledgable agent that helps me create other agents. Can you do that?"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance. The AI offers help and the human requests information on emojis. The AI explains that emojis are digital images used to convey an idea or emotion, often used in text messages and social media. Emojis come in categories such as facial expressions and food, and the AI offers to provide specific examples.
Human: Cool, can you print your favorites?
AI: Unfortunately, as an AI language model, I do not have personal favorites. However, some commonly used emojis include the laughing face with tears, heart eyes, thumbs up, and the crying face. Is there a specific category or emotion you would l

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 37d3d1feb88b0406cac7e287816340fb in your message.).



> Finished chain.
Absolutely! I can definitely pretend to be an agent that helps you create other agents. What kind of agents are you interested in creating? Do you have any specific goals or functions in mind for these agents? Let's start brainstorming together!


In [ ]:
print(conversation.run(input="I'd like to create an agent for my female partner to interact with, to help her explore some of her fantasies. Can you help me?"))




> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance with emojis. The AI explains that emojis are used to convey ideas and emotions in digital messages and offers to provide specific examples. The human asks for some popular emojis and the AI lists some commonly used ones, such as the laughing face with tears and the red heart. The human asks if the AI can write a poem about emojis, and the AI agrees and quickly produces a poem about the power of emojis to convey meaning in text messages.
Human: Hahaha. Can you play a game with me?
AI: Sure, what game would you like to play? As an AI language model, I can play games like trivia, word assoc

In [ ]:
print(conversation.run(input="Yes, all that sounds good. I'd like to use an LLM to generate the agent's responses. I think i need most help is with the prompts to use. Can you help me with that?"))





> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
The following is a friendly texting conversation between a Human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, tells the human as such.

Please use markdown in your output format.

Human: The human greets the AI and asks for assistance with emojis. The AI explains that emojis are used to convey ideas and emotions in digital messages and offers to provide specific examples. The human asks for some popular emojis and the AI lists some commonly used ones, such as the laughing face with tears and the red heart. The human asks if the AI can write a poem about emojis, and the AI agrees and quickly produces a poem about the power of emojis to convey meaning in text messages.
Human: Hahaha. Can you play a game with me?
AI: Sure, what game would you like to play? As an AI language model, I can play games like trivia, word assoc

In [ ]:
print(conversation.run(input="That sounds great. Let's start with something simple. She is bisexual, and into gender nonconforming people sometimes. She likes to feel safe and cared for, but at the same time free to explore her sexuality her own way. "))


ReadTimeout: The read operation timed out

In [ ]:
print(conversation.run(input=""))


In [ ]:
print(conversation.run(input=""))


In [ ]:
print(conversation.run(input=""))


In [ ]:
PREFIX = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:"""
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```"""

SUFFIX = """Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""